In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# import os
# os.chdir('/content/drive/My Drive/translate')

In [ ]:
drive.flush_and_unmount()

In [ ]:
!pip install sacrebleu transformers sentencepiece

In [ ]:
import tensorflow as tf
import numpy as np
import json
from sacrebleu import BLEU
import transformers

In [ ]:
bleu = BLEU()
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

checkpoint = 'google/mt5-small'
model_name = 'mT5_small_PTT_2023_08_06_Thu_Aug_17_081645_2023.ckpt'
split_char = '⫯'

datasets_name = 'PTT_2023_08_06'
prefix = "translate engTyping to Traditional Chinese:".split()

In [ ]:
model = transformers.TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model.load_weights(f'models/{model_name}')
tokenizer = transformers.AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
with open(f'datasets/{datasets_name}_engTyping_inserted_lines.txt', encoding='utf8') as file:
    engTyping_inserted_lines = file.readlines()
with open(f'datasets/{datasets_name}_zh_lines.txt', encoding='utf8') as file:
    zh_lines = file.readlines()
lines_len = len(engTyping_inserted_lines)
assert lines_len == len(zh_lines)
engTyping_inserted_lines = engTyping_inserted_lines[int(lines_len * 0.85):]
engTyping_lines = [prefix + line.split(split_char) for line in engTyping_inserted_lines]
zh_lines = zh_lines[int(lines_len * 0.85):]
for i in range(len(zh_lines)):
    zh_lines[i] = ' '.join(zh_lines[i].split(split_char)[1:-1])

In [ ]:
pred_sentences = []
loop_times = 300
eng_len = len(engTyping_lines)
split_point = eng_len // loop_times
for k in range(loop_times + 1):
    lines = engTyping_lines[k * split_point:(k + 1) * split_point]
    len_lines = len(lines)
    inputs = tokenizer(lines, is_split_into_words=True, return_tensors="tf", truncation=True, padding=True).input_ids
    outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
    decoded_sentences = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    pred_sentences.extend(decoded_sentences)
    print(f'\r{k}/{loop_times}', end='')


print(pred_sentences)

299/300

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
result = bleu.corpus_score(pred_sentences, [zh_lines])
print(result)

BLEU = 83.17 92.7/86.9/81.4/76.2 (BP = 0.989 ratio = 0.989 hyp_len = 2878258 ref_len = 2909826)


In [ ]:
len(pred_sentences)

231813

In [ ]:
len(zh_lines)

231813

In [ ]:
import pickle
with open('translator_mT5_lines.pkl', 'wb') as file:
    pickle.dump(pred_sentences, file)